In [1]:
#python_bitbankccのパッケージをインポート
#cloud9で起動するときのコマンド
#jupyter notebook --ip $IP --port $PORT --no-browser
import python_bitbankcc 
import datetime
import os 
import time
import numpy as np
import pandas as pd
import sys
from dateutil.relativedelta import relativedelta
#トレンドラインを引くため
from scipy.stats import linregress
#正規化
from sklearn.preprocessing import MinMaxScaler

In [2]:
#機械学習用のモジュール
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from sklearn import linear_model
% matplotlib inline
from __future__ import print_function
import copy
import matplotlib
matplotlib.style.use('ggplot')

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
# public API classのオブジェクトを取得
pub = python_bitbankcc.public()

In [4]:
#APIから基本データの取得、dataframeへ挿入
def get_candle(trade_name,span,back_day):
    # ロウソク足データを取得
    pub = python_bitbankcc.public()
    value = pub.get_candlestick( trade_name,span, back_day )
    candle = value['candlestick'][0]
    #print(candle['ohlcv'][0])
    dataframe = pd.DataFrame(candle['ohlcv'],columns=["Open", "High","Low","Close","Volume","Timestamp"])
    return dataframe


In [5]:
#指定した日から今日までの基本データをdataframeにして取得
#back_day,todayはdatetime
# span = ['1min', '5min', '15min', '30min', '1hour', '4hour', '8hour', '12hour', '1day', '1week']
#trade_name = ['btc_jpy', 'xrp_jpy', 'ltc_btc', 'eth_btc', 'mona_jpy', 'mona_btc', 'bcc_jpy', 'bcc_btc']

def make_df(trade_name,span,back_day,today):
    i = 1
    if (span =='1min')or(span== '5min')or(span== '15min')or(span== '30min')or(span== '1hour'):
        #基準が９時なので、そこを合わせてあげる
        if 0 <= today.hour < 9:
            k = 1
        elif 9 <= today.hour <= 23:
            k = 0

        while back_day <= today - datetime.timedelta(days=k):
            if i == 1:
                df1 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y%m%d'))
                back_day = back_day + datetime.timedelta(days=1)
                #print(back_day,len(df1))
                i += 1
            else:
                df2 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y%m%d'))
                df1 = pd.concat([df1, df2])
                back_day = back_day + datetime.timedelta(days=1)
                #print(back_day,len(df1))
                i += 1
    else:
        
        today = datetime.date(today.year, today.month, today.day)
        back_day = datetime.date(back_day.year, back_day.month, back_day.day)


        while back_day <= today:
            if i == 1:
                df1 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y'))
                back_day = back_day + relativedelta(years=1)
                i += 1
            else:
                df2 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y'))
                df1 = pd.concat([df1, df2])
                back_day = back_day +  relativedelta(years=1)
                i += 1
            

    return df1



In [6]:
def read_date(x):
    return datetime.datetime.fromtimestamp(x/1000)


In [7]:

# 単純移動平均（SMA）を取得する関数
#上記のdfデータフレーム
#numいくつの平均を求めるか
def getMA( df,num ):
 
    tmp = []
    avg = np.array([])
    for i in range(len(df) - num + 1):
        for j in range(num):
            #print(df['Close'][i+j])
            tmp.append( df['Close'][i+j])
            
         # 平均値計算
        value = np.average(tmp)
        avg = np.append(avg,value)
        tmp = []
    
    return avg

In [8]:
#ゴールデン・デッドクロス判定
def golden_dead(data_df1):
    golden=np.zeros(len(data_df1['Cross']),dtype=int)
    dead=np.zeros(len(data_df1['Cross']),dtype=int)
    for i in range(len(data_df1['Cross'])-1):
        x = data_df1['Cross'][i]
        y = data_df1['Cross'][i+1]
        if ((x <= 0) & (y<=0)) |((x >= 0) & (y>=0)):
            pass
        elif ((x <= 0) & (y>=0)) :
            golden=np.insert(golden,i+1,1)
            golden=np.delete(golden,i+2)
        elif ((x >= 0) & (y<=0)) :
            dead=np.insert(dead,i+1,1)
            dead=np.delete(dead,i+2)

    return golden,dead
    

In [9]:
#上下判定
def up_down(data_df1):
    up=np.zeros(len(data_df1['Close']),dtype=int)
    mid_up=np.zeros(len(data_df1['Close']),dtype=int)
    down=np.zeros(len(data_df1['Close']),dtype=int)
    mid_down=np.zeros(len(data_df1['Close']),dtype=int)
    for i in range(len(data_df1['Close'])-1):
        x = data_df1['Close'][i]
        y = data_df1['Close'][i+1]
#         if (0<=(y-x)<=0.5):
#             mid_up=np.insert(mid_up,i+1,1)
#             min_up=np.delete(mid_up,i+2)
#         elif (-0.5<=(y-x)<0):
#             mid_down=np.insert(mid_down,i+1,1)
#             min_down=np.delete(mid_down,i+2)
        if ((y-x)>0):
            up=np.insert(up,i+1,1)
            up=np.delete(up,i+2)
        elif ((y-x)<=0) :
            down=np.insert(down,i+1,1)
            down=np.delete(down,i+2)

    return up,down

In [10]:
#highトレンドラインを作成する関数
#df データフレーム
#num どれだけ前から
def make_high_trend_line(df,num):
    slope= np.zeros(num-1,dtype=float)
    intercept = np.zeros(num-1,dtype=float)
    for i in range(len(df)-num+1):        
        df_fin = df.copy()
        df_fin = df_fin[i:i+num]
        df_high = df.copy()
        df_high = df_high[i:i+num]
    
        # 高値のトレンドライン
        while len(df_high)>3:
            reg_1 = linregress(x=df_high['index'],y=df_high['High'])
            df_high  =  df_high.loc[df_high['High']>reg_1[0]*df_high['index']+reg_1[1]]

        if len(df_high)<=1:
            pass
        else:
            reg_1 = linregress(x=df_high['index'],y=df_high['High'])
        
        slope = np.append(slope,reg_1[0])
        intercept  = np.append(intercept ,reg_1[1])
    return slope,intercept    

In [11]:
#lowトレンドラインを作成する関数
#df データフレーム
#num どれだけ前から
def make_low_trend_line(df,num):
    slope= np.zeros(num-1,dtype=float)
    intercept = np.zeros(num-1,dtype=float)
    for i in range(len(df)-num+1):        
        df_fin = df.copy()
        df_fin = df_fin[i:i+num]
        df_low = df.copy()
        df_low = df_low[i:i+num]

    # 安値のトレンドライン
        while len(df_low)>3: 
            reg_2 = linregress( x = df_low['index'], y = df_low['Low']   )
            df_low = df_low.loc[df_low['Low'] < reg_2[0] * df_low['index'] + reg_2[1]]

        if len(df_low)<=1:
            pass
        else:
            reg_2 = linregress(x = df_low['index'],y = df_low['Low'])
            
        slope = np.append(slope,reg_2[0])
        intercept  = np.append(intercept ,reg_2[1])
    return slope,intercept    

In [12]:
#slope,傾き　intercept,切片　df、データフレーム
def make_trend_value(slope,intercept,df):
    index = np.array(df['index'])
    trend_value = slope*index + intercept
    return trend_value

In [13]:
#関数の使い方
today = datetime.datetime.today() 
back_day = today - datetime.timedelta(days=200)
trade_name = 'xrp_jpy'
span = '1hour'
num = 5

In [14]:
#基本データの取得
old = time.time()
data_df1 =  make_df(trade_name,span,back_day,today)
data_df1['Timestamp'] = data_df1['Timestamp'].apply(read_date)
#indexの割り振り
data_df1 = data_df1.reset_index()
#df1の値は全てstrなのでTimestamp以外floatにキャストする
#Timestampは文字列のままでいいので削除することを指定
execlude = ['Timestamp']
cast = [col for col in data_df1.columns if col not in execlude]
for item in cast:
    data_df1 = data_df1.astype({item: float})

print(time.time()-old)
print(data_df1.tail())

52.04891037940979
      index    Open    High     Low   Close        Volume           Timestamp
4810   10.0  46.702  46.750  46.498  46.583  3.510023e+06 2018-10-14 19:00:00
4811   11.0  46.593  46.839  46.542  46.754  2.647676e+06 2018-10-14 20:00:00
4812   12.0  46.754  47.100  46.650  46.930  6.230422e+06 2018-10-14 21:00:00
4813   13.0  46.930  46.979  46.500  46.670  4.468723e+06 2018-10-14 22:00:00
4814   14.0  46.669  46.760  46.625  46.681  1.104098e+06 2018-10-14 23:00:00


In [15]:
#移動平均線np
MA5 = getMA( data_df1,num)
MA25 = getMA( data_df1,num+20)
zero5 = np.zeros(num-1,dtype = float )
zero25 = np.zeros(num+20-1,dtype = float )
MA5 = np.insert(MA5, 0, zero5)
MA25 = np.insert(MA25, 0, zero25)
print(MA25,MA5)
print(MA25.shape,type(MA25),MA5.shape,type(MA5))

[ 0.       0.       0.      ... 46.71832 46.71144 46.71468] [ 0.      0.      0.     ... 46.7898 46.7282 46.7236]
(4815,) <class 'numpy.ndarray'> (4815,) <class 'numpy.ndarray'>


In [16]:
#自作変数を作成する
data_df1['High-Low']=data_df1['High']-data_df1['Low']
data_df1['Close-Open']=data_df1['Close']-data_df1['Open']
data_df1['Similarity'] = data_df1['Close-Open'] / (data_df1['High-Low'] + 0.000001)
data_df1['(High-Low)*Volume '] = data_df1['High-Low']*data_df1['Volume']
data_df1['(Close-Open)*Volume '] = data_df1['Close-Open']*data_df1['Volume']
data_df1['High-Low']
data_df1['MA5'] = MA5
data_df1['MA25'] = MA25
data_df1['Cross'] = MA5 - MA25
data_df1['index'] = data_df1.index + 1

In [17]:
data_df1['golden'],data_df1['dead']  = golden_dead(data_df1)

In [18]:
old1 = time.time()
slope1,intercept1 = make_high_trend_line(df=data_df1,num=32)
print(time.time()-old1)

38.24955487251282


In [19]:
old1 = time.time()
slope2,intercept2 = make_low_trend_line(df=data_df1,num=32)
print(time.time()-old1)

40.49586510658264


In [20]:
tr_high = make_trend_value(slope1,intercept1,data_df1)
tr_low  = make_trend_value(slope2,intercept2,data_df1)

In [21]:
data_df1['high_slope'] = slope1
data_df1['low_slope'] = slope2
data_df1['tr_high'] = tr_high
data_df1['tr_low'] = tr_low
data_df1['tr_mid'] = (tr_high + tr_low)/2

In [22]:
#data_df1['up'],data_df1['mid_up'],data_df1['min_down'],data_df1['down'] = up_down(data_df1)
# a,b = up_down(data_df1)
# print(len(a),len(b))
data_df1['up'],data_df1['down'] = up_down(data_df1)


In [23]:
#data作成までの時間
print(time.time()-old)
#作成変数の表示
data_df1.loc[len(data_df1)-9:]

134.3154320716858


,index,Open,High,Low,Close,Volume,Timestamp,High-Low,Close-Open,Similarity,...,Cross,golden,dead,high_slope,low_slope,tr_high,tr_low,tr_mid,up,down
4806,4807,46.950,47.180,46.921,46.995,3.058577e+06,2018-10-14 15:00:00,0.259,0.045,0.173745,...,0.30028,0,0,0.004859,0.008232,47.612646,46.225460,46.919053,1,0
4807,4808,46.995,47.099,46.796,47.003,2.714953e+06,2018-10-14 16:00:00,0.303,0.008,0.026403,...,0.34388,0,0,0.003984,0.008232,47.608087,46.233692,46.920890,1,0
4808,4809,47.025,47.094,46.900,46.978,1.984632e+06,2018-10-14 17:00:00,0.194,-0.047,-0.242267,...,0.28636,0,0,0.003984,0.008232,47.612071,46.241924,46.926998,0,1
4809,4810,46.978,47.039,46.506,46.704,4.698911e+06,2018-10-14 18:00:00,0.533,-0.274,-0.514070,...,0.16844,0,0,0.003984,0.051000,47.616055,46.879667,47.247861,0,1
4810,4811,46.702,46.750,46.498,46.583,3.510023e+06,2018-10-14 19:00:00,0.252,-0.119,-0.472220,...,0.11784,0,0,0.003984,0.051000,47.620039,46.930667,47.275353,0,1
4811,4812,46.593,46.839,46.542,46.754,2.647676e+06,2018-10-14 20:00:00,0.297,0.161,0.542086,...,0.08356,0,0,0.003984,0.051000,47.624023,46.981667,47.302845,1,0
4812,4813,46.754,47.100,46.650,46.930,6.230422e+06,2018-10-14 21:00:00,0.450,0.176,0.391110,...,0.07148,0,0,0.003984,0.051000,47.628007,47.032667,47.330337,1,0
4813,4814,46.930,46.979,46.500,46.670,4.468723e+06,2018-10-14 22:00:00,0.479,-0.260,-0.542796,...,0.01676,0,0,0.003984,0.019894,47.631991,46.499354,47.065673,0,1
4814,4815,46.669,46.760,46.625,46.681,1.104098e+06,2018-10-14 23:00:00,0.135,0.012,0.088888,...,0.00892,0,0,0.002722,0.019894,47.622222,46.519248,47.070735,1,0


In [24]:
#次のCloseを予測する機械学習
#使わない変数を除外
exe_cols = ['index','Timestamp']
feature_cols = [col for col in data_df1.columns if col not in exe_cols]
print(feature_cols)
#機械学習用にnp配列に変換
data_np= np.array(data_df1[feature_cols])
print(data_np.shape)

['Open', 'High', 'Low', 'Close', 'Volume', 'High-Low', 'Close-Open', 'Similarity', '(High-Low)*Volume ', '(Close-Open)*Volume ', 'MA5', 'MA25', 'Cross', 'golden', 'dead', 'high_slope', 'low_slope', 'tr_high', 'tr_low', 'tr_mid', 'up', 'down']
(4815, 22)


In [25]:
Y = np.array(data_df1['Close'])#1次元
Y = np.delete(Y,0)
print(Y,Y.shape)
X = data_np[:len(Y),0:]
print(X[:],X.shape)

[60.83  60.945 61.12  ... 46.93  46.67  46.681] (4814,)
[[61.1        61.5        60.501      ...  0.          0.
   0.        ]
 [60.698      61.3        60.62       ...  0.          1.
   0.        ]
 [60.83       61.         60.551      ...  0.          1.
   0.        ]
 ...
 [46.593      46.839      46.542      ... 47.302845    1.
   0.        ]
 [46.754      47.1        46.65       ... 47.33033698  1.
   0.        ]
 [46.93       46.979      46.5        ... 47.06567274  0.
   1.        ]] (4814, 22)


In [26]:
# 学習では、29/30を使うものとします。これは情報の偏りを防ぐためのものであり、全体でも構いません
L = int(len(X)//(200/199))
print(L)
train_x = X[50:L,:]
train_y = Y[50:L]

4789


In [27]:
# 残りの全てをテストデータとします
test_x = X[L:len(X),:]
test_y = Y[L:len(Y)]

In [28]:
#正規化してみる
df_2 = data_df1[:len(Y)]
del df_2['index']
del df_2['Timestamp']
print(len(Y),len(df_2))
df_2['Close_tomorrow'] = Y
print(df_2[len(df_2)-9:])
# データセットのサイズを確認
print(df_2.shape[0])
print(df_2.shape[1])
n = df_2.shape[0]
p = df_2.shape[1]
 
# 訓練データとテストデータへ切り分け
train_start = 30
train_end = int(np.floor(0.95*n))
test_start = train_end + 1
test_end = n
data_train = df_2[train_start: train_end]
data_test = df_2[test_start:test_end]

# データの正規化
scaler = MinMaxScaler()
print(len(data_train))
scaler.fit(data_train)
data_train_norm = scaler.transform(data_train)
data_test_norm = scaler.transform(data_test)
print(data_train_norm.shape,data_test_norm.shape)

# 特徴量とターゲットへ切り分け
print(type(data_train_norm))
train_x = data_train_norm[:,:data_train_norm.shape[1]-1]
train_y = data_train_norm[:,data_train_norm.shape[1]-1:]
test_x = data_test_norm[:, :data_train_norm.shape[1]-1]
test_y = data_test_norm[:, data_train_norm.shape[1]-1:]
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)


4814 4814
        Open    High     Low   Close        Volume  High-Low  Close-Open  \
4805  47.312  47.408  46.914  46.950  5.867135e+06     0.494      -0.362   
4806  46.950  47.180  46.921  46.995  3.058577e+06     0.259       0.045   
4807  46.995  47.099  46.796  47.003  2.714953e+06     0.303       0.008   
4808  47.025  47.094  46.900  46.978  1.984632e+06     0.194      -0.047   
4809  46.978  47.039  46.506  46.704  4.698911e+06     0.533      -0.274   
4810  46.702  46.750  46.498  46.583  3.510023e+06     0.252      -0.119   
4811  46.593  46.839  46.542  46.754  2.647676e+06     0.297       0.161   
4812  46.754  47.100  46.650  46.930  6.230422e+06     0.450       0.176   
4813  46.930  46.979  46.500  46.670  4.468723e+06     0.479      -0.260   

      Similarity  (High-Low)*Volume   (Close-Open)*Volume        ...        \
4805   -0.732792        2.898365e+06         -2.123903e+06       ...         
4806    0.173745        7.921715e+05          1.376360e+05       ...     

C:\Users\ibuki\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
test_y = test_y.reshape(test_y.shape[0], 1)
test_inv = np.concatenate(( test_x,test_y), axis=1)
test_inv = scaler.inverse_transform(test_inv)

In [30]:
train_y=train_y.ravel()

In [31]:
rf = RandomForestRegressor(random_state=1234)

In [32]:
params = {'n_estimators': [15,20,25], 'max_depth': [5,17,30]}
gscv = GridSearchCV(rf, param_grid=params, verbose=1,
                    cv=3, scoring='neg_mean_squared_error')
gscv.fit(train_x, train_y)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   15.5s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1234, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [15, 20, 25], 'max_depth': [5, 17, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [33]:
a = gscv.best_params_

In [34]:
print(a)
n_estimators = a.get('n_estimators')
max_depth = a.get('max_depth')

{'max_depth': 30, 'n_estimators': 25}


In [35]:
rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,random_state=1234)
rf.fit(train_x, train_y)
y_pred_rf = rf.predict(test_x)
rf_mse = mean_squared_error(test_y, y_pred_rf)
print('RandomForest MSE: ', rf_mse)

RandomForest MSE:  9.39513385070395e-05


In [36]:
# 予測値をテストデータに戻そう（値も正規化からインバース）
print(test_x.shape,y_pred_rf.T.shape,type(y_pred_rf))
a = y_pred_rf.T.reshape(y_pred_rf.T.shape[0],1)
pred_tested = np.concatenate(( test_x,a), axis=1)
pred_inv = scaler.inverse_transform(pred_tested)
print(test_inv-pred_inv)

(240, 22) (240,) <class 'numpy.ndarray'>
[[ 0.          0.          0.         ...  0.          0.
   0.31593111]
 [ 0.          0.          0.         ...  0.          0.
  -0.23191867]
 [ 0.          0.          0.         ...  0.          0.
  -0.14318581]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.55822267]
 [ 0.          0.          0.         ...  0.          0.
   0.17098267]
 [ 0.          0.          0.         ...  0.          0.
   0.38768667]]


In [37]:
y_pred_rf = pred_inv[:,pred_inv.shape[1]-1:]
test_y = test_inv[:,test_inv.shape[1]-1:]


In [38]:
# 予測結果を出力します。これはランダムフォレスト
result = pd.DataFrame(y_pred_rf)
result.columns = ['y_pred_rf']
result['test_y'] = test_y
#２乗平均誤差ではなく、ただの差
result['RMS'] = (result['test_y']-result['y_pred_rf'])
print(result.loc[:])

     y_pred_rf  test_y       RMS
0    59.983069  60.299  0.315931
1    60.470919  60.239 -0.231919
2    60.543186  60.400 -0.143186
3    60.511207  60.570  0.058793
4    60.664114  60.572 -0.092114
5    60.559127  60.501 -0.058127
6    60.500160  60.149 -0.351160
7    60.084283  60.195  0.110717
8    60.320544  60.018 -0.302544
9    59.969123  60.239  0.269877
10   60.510227  59.811 -0.699227
11   59.733438  59.456 -0.277438
12   59.489166  59.403 -0.086166
13   59.459861  59.447 -0.012861
14   59.409223  59.540  0.130777
15   59.396755  59.514  0.117245
16   59.450164  58.890 -0.560164
17   59.134862  58.362 -0.772862
18   58.643202  58.848  0.204798
19   59.069340  58.432 -0.637340
20   58.778322  58.840  0.061678
21   59.017147  58.589 -0.428147
22   58.713213  58.700 -0.013213
23   58.542683  58.967  0.424317
24   58.965113  58.550 -0.415113
25   58.754821  58.407 -0.347821
26   58.664756  58.010 -0.654756
27   58.297140  58.102 -0.195140
28   58.118713  57.978 -0.140713
29   58.11

In [39]:
# Plot拡大
plt.plot(range(0,len(result[0:24])), test_y[len(test_y)-24:], label='Actual price', color='blue', marker = 'o')
plt.plot(range(0,len(result[0:24])), y_pred_rf[len(test_y)-24:], label='Predicted price', color='red', marker ='x')
plt.xlabel(span)
plt.ylabel('Price (\)')
n3='{0} Price by RandomForest'.format(trade_name)
plt.title(n3)
plt.grid(True)
plt.legend()
n4='{0}2 by RandomForest.png'.format(trade_name)
plt.savefig(n4)
plt.close()
plt.show()

In [40]:
# 変数増加法を実行する関数
def get_gfs_feature_indices(X, y, features, clf):
    X_train_, X_test_, y_train_, y_test_ = \
        train_test_split(X, y, test_size=0.3, shuffle=False)
    feature_indices = {feature: idx for idx, feature in enumerate(features)}
    features = set(features)
    last_mse = np.inf
    chosen_features = set()
    while len(chosen_features) < len(features):
        mse_features = []
        for feature in (features - chosen_features):
            candidates = chosen_features.union(set([feature]))
            indices = [feature_indices[feature] for feature in candidates]
            clf.fit(X_train_[:, indices], y_train_)
            y_pred = clf.predict(X_test_[:, indices])
            mse = mean_squared_error(y_test_, y_pred)
            mse_features += [(mse, feature)]
        mse, feature = min(mse_features)
        if mse >= last_mse:
            break
        last_mse = mse
        print('Newly Added Feature: {},\tMSE Score: {}'.format(feature, mse))
        chosen_features.add(feature)
    return [feature_indices[feature] for feature in chosen_features]

In [41]:
# 上記関数を使用して変数増加法を実行し、MSEを算出
#feature_cols = list('abcdefghij')
feature_cols = list('abcdefghijklmnopqrstuv')
print(feature_cols,len(feature_cols))

selected_feature_index_by_RandomForestRegressor = get_gfs_feature_indices(X=train_x,y=train_y,features=feature_cols,clf= RandomForestRegressor())
print(selected_feature_index_by_RandomForestRegressor)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v'] 22
Newly Added Feature: d,	MSE Score: 0.0016400158352345734
Newly Added Feature: o,	MSE Score: 0.0016304403380237014
Newly Added Feature: n,	MSE Score: 0.00159752029419611
[3, 14, 13]


In [42]:
#ランダムフォレスト
rf = RandomForestRegressor(random_state=1234)
rf.fit(train_x[:, selected_feature_index_by_RandomForestRegressor], train_y)
y_pred_rf = rf.predict(test_x[:, selected_feature_index_by_RandomForestRegressor])
rf_mse = mean_squared_error(test_y, y_pred_rf)
print('RandomForest MSE: ', rf_mse)

RandomForest MSE:  2718.3645572519486


In [43]:
params = {'n_estimators': [15,20,25], 'max_depth': [6,10,20]}
gscv = GridSearchCV(rf, param_grid=params, verbose=1,
                    cv=3, scoring='neg_mean_squared_error')
gscv.fit(train_x[:, selected_feature_index_by_RandomForestRegressor], train_y)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    1.8s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1234, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [15, 20, 25], 'max_depth': [6, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [44]:
a = gscv.best_params_

In [45]:
print(a)
n_estimators = a.get('n_estimators')
max_depth = a.get('max_depth')

{'max_depth': 10, 'n_estimators': 15}


In [46]:
rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,random_state=1234)
rf.fit(train_x[:, selected_feature_index_by_RandomForestRegressor], train_y)
y_pred_rf_gfs = rf.predict(test_x[:, selected_feature_index_by_RandomForestRegressor])
rf_mse = mean_squared_error(test_y, y_pred_rf_gfs)
print('RandomForest MSE: ', rf_mse)

RandomForest MSE:  2718.38982855276


In [47]:
# 予測値をテストデータに戻そう（値も正規化からインバース）
print(test_x.shape,y_pred_rf_gfs.T.shape,type(y_pred_rf_gfs))
b=y_pred_rf_gfs.T.reshape(y_pred_rf_gfs.T.shape[0],1)
pred_tested = np.concatenate(( test_x,b), axis=1)
pred_inv = scaler.inverse_transform(pred_tested)
print(test_inv-pred_inv)

(240, 22) (240,) <class 'numpy.ndarray'>
[[ 0.          0.          0.         ...  0.          0.
   0.25150951]
 [ 0.          0.          0.         ...  0.          0.
  -0.04011297]
 [ 0.          0.          0.         ...  0.          0.
  -0.02801902]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.09460056]
 [ 0.          0.          0.         ...  0.          0.
  -0.17473278]
 [ 0.          0.          0.         ...  0.          0.
   0.21463556]]


In [48]:
y_pred_rf_gfs = pred_inv[:,pred_inv.shape[1]-1:]
test_y = test_inv[:,test_inv.shape[1]-1:]


In [49]:
# 予測結果を出力します。これはランダムフォレスト
result = pd.DataFrame(y_pred_rf_gfs)
result.columns = ['y_pred_rf_gfs']
result['test_y'] = test_y
#２乗平均誤差ではなく、ただの差
result['RMS'] = (result['test_y']-result['y_pred_rf_gfs'])
print(result.loc[:])

     y_pred_rf_gfs  test_y       RMS
0        60.047490  60.299  0.251510
1        60.279113  60.239 -0.040113
2        60.428019  60.400 -0.028019
3        60.500030  60.570  0.069970
4        60.532285  60.572  0.039715
5        60.532285  60.501 -0.031285
6        60.485040  60.149 -0.336040
7        59.938227  60.195  0.256773
8        59.621875  60.018  0.396125
9        59.938227  60.239  0.300773
10       60.428019  59.811 -0.617019
11       60.257180  59.456 -0.801180
12       59.384158  59.403  0.018842
13       59.301704  59.447  0.145296
14       59.384158  59.540  0.155842
15       59.467226  59.514  0.046774
16       59.467226  58.890 -0.577226
17       58.981481  58.362 -0.619481
18       58.366921  58.848  0.481079
19       58.882371  58.432 -0.450371
20       58.289590  58.840  0.550410
21       58.996133  58.589 -0.407133
22       59.669185  58.700 -0.969185
23       59.055653  58.967 -0.088653
24       59.238752  58.550 -0.688752
25       59.659058  58.407 -1.252058
2

In [50]:
# Plot拡大
plt.plot(range(0,len(result[0:24])), test_y[len(test_y)-24:], label='Actual price', color='blue', marker = 'o')
plt.plot(range(0,len(result[0:24])), y_pred_rf_gfs[len(test_y)-24:], label='Predicted price', color='red', marker ='x')
plt.xlabel(span)
plt.ylabel('Price (\)')
n3='{0} Price by RandomForest gfs'.format(trade_name)
plt.title(n3)
plt.grid(True)
plt.legend()
n4='{0}2 by RandomForest gfs.png'.format(trade_name)
plt.savefig(n4)
plt.close()
plt.show()